In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826948e+12,  1.05165000e+05,  1.04909800e+05, ...,
         1.04768000e+05,  4.98372300e+03, -1.00000000e+00],
       [ 1.73827092e+12,  1.04909800e+05,  1.05189300e+05, ...,
         1.04737000e+05,  4.83362100e+03, -1.00000000e+00],
       [ 1.73827926e+12,  1.05189200e+05,  1.04416800e+05, ...,
         1.04356100e+05,  4.86977900e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([21295, 23208]))
(array([0, 1]), array([4207, 4694]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-17 01:19:11,327] A new study created in memory with name: no-name-3141fbac-6736-493b-b4e4-4ac6149bd7d5
[I 2025-03-17 01:19:27,978] Trial 0 finished with value: 0.8586345276994365 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 122, 'max_depth': 114, 'min_gain_to_split': 0.20906764959451943, 'min_data_in_leaf': 218, 'lambda_l1': 20.289696478687762, 'lambda_l2': 91.70018723363222, 'num_boost_round': 426}. Best is trial 0 with value: 0.8586345276994365.


precision: 0.7970914127423823, recall: 0.7356199403493822, f1: 0.7651229780633725


[I 2025-03-17 01:19:46,885] Trial 1 finished with value: 0.8595671952593062 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 220, 'max_depth': 143, 'min_gain_to_split': 0.14398397194424173, 'min_data_in_leaf': 148, 'lambda_l1': 42.132918857711644, 'lambda_l2': 20.514233676632312, 'num_boost_round': 1274}. Best is trial 1 with value: 0.8595671952593062.


precision: 0.7973588342440802, recall: 0.746058798466127, f1: 0.7708562623816861


[I 2025-03-17 01:20:04,220] Trial 2 finished with value: 0.8581580661362477 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 288, 'max_depth': 791, 'min_gain_to_split': 0.883812776949132, 'min_data_in_leaf': 387, 'lambda_l1': 24.00890167179731, 'lambda_l2': 28.07157269590471, 'num_boost_round': 1171}. Best is trial 1 with value: 0.8595671952593062.


...

precision: 0.7838723776223776, recall: 0.764167021729868, f1: 0.7738942826321467


[I 2025-03-17 02:04:00,062] Trial 98 finished with value: 0.8622046725743376 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 105, 'max_depth': 406, 'min_gain_to_split': 0.8759027853241006, 'min_data_in_leaf': 276, 'lambda_l1': 20.7213620064041, 'lambda_l2': 53.013791376982375, 'num_boost_round': 682}. Best is trial 80 with value: 0.864762748068657.


precision: 0.7895897662108513, recall: 0.7626757562846187, f1: 0.775899436497616


[I 2025-03-17 02:04:26,197] Trial 99 finished with value: 0.8628883992218217 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 295, 'max_depth': 540, 'min_gain_to_split': 0.378721467390038, 'min_data_in_leaf': 348, 'lambda_l1': 17.42421664091401, 'lambda_l2': 40.1854701778451, 'num_boost_round': 909}. Best is trial 80 with value: 0.864762748068657.


precision: 0.7905164737310775, recall: 0.7564976565828717, f1: 0.7731330285216634


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 125,
 'max_depth': 410,
 'min_gain_to_split': 0.8375539513793376,
 'min_data_in_leaf': 254,
 'lambda_l1': 13.749730556155288,
 'lambda_l2': 68.57526638592569,
 'num_boost_round': 984}

In [7]:
params = {
    # "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [13]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [14]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8598716870628407


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [15]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_1h.parquet")
meta_label = np.load("data/label_meta_1h.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_comb_spectrum_pwr_14         0
1h_comb_spectrum_pwr_13         0
1h_comb_spectrum_pwr_12         0
1h_comb_spectrum_pwr_11         0
side_model_res                  0
Length: 11842, dtype: int64

In [2]:
df_features = df_features[364:]
meta_label = meta_label[364:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(44503, 11842)
(44503, 7)


In [3]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

np.unique(meta_label_test, return_counts=True)

(44503, 2368)


(array([0, 1]), array([2284, 6617]))

In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-16 12:22:16,964] A new study created in memory with name: no-name-90731219-4045-41f3-ac75-15bf9a24d28c
[I 2025-03-16 12:22:37,547] Trial 0 finished with value: 0.801057801284473 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 108, 'max_depth': 905, 'min_gain_to_split': 0.2753429852815696, 'min_data_in_leaf': 245, 'lambda_l1': 33.06194186122965, 'lambda_l2': 51.523104419141724, 'num_boost_round': 610}. Best is trial 0 with value: 0.801057801284473.


precision: 0.8009972801450589, recall: 0.801118331570198, f1: 0.801057801284473, auc: 0.7097699445809988, fbeta: 0.7041731536566697


[I 2025-03-16 12:23:21,952] Trial 1 finished with value: 0.7902341033763425 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 441, 'max_depth': 408, 'min_gain_to_split': 0.5236577050409843, 'min_data_in_leaf': 259, 'lambda_l1': 19.899624861030844, 'lambda_l2': 10.229699461031052, 'num_boost_round': 610}. Best is trial 0 with value: 0.801057801284473.


precision: 0.8084097375908947, recall: 0.7728577905395194, f1: 0.7902341033763425, auc: 0.712599320277574, fbeta: 0.6995468366728085


[I 2025-03-16 12:23:35,026] Trial 2 finished with value: 0.5603874907865641 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 459, 'max_depth': 903, 'min_gain_to_split': 0.6267506891485707, 'min_data_in_leaf': 170, 'lambda_l1': 48.28615982867741, 'lambda_l2': 5.385175611890521, 'num_boost_round': 478}. Best is trial 0 with value: 0.801057801284473.


...

precision: 0.8019559902200489, recall: 0.7931086595133746, f1: 0.7975077881619937, auc: 0.7072010029889049, fbeta: 0.7018381731029555


[I 2025-03-16 14:23:44,446] Trial 67 finished with value: 0.8146904205607477 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 436, 'max_depth': 681, 'min_gain_to_split': 0.1526837617540002, 'min_data_in_leaf': 254, 'lambda_l1': 17.52131857476765, 'lambda_l2': 10.856351424819788, 'num_boost_round': 1951}. Best is trial 44 with value: 0.8275178710453677.


precision: 0.7881056646418986, recall: 0.8431313283965544, f1: 0.8146904205607477, auc: 0.7073773385804806, fbeta: 0.7053890403286264


[W 2025-03-16 14:24:04,945] Trial 68 failed with parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 482, 'max_depth': 586, 'min_gain_to_split': 0.08589670894303576, 'min_data_in_leaf': 236, 'lambda_l1': 7.844852626066834, 'lambda_l2': 6.571404199302288, 'num_boost_round': 1332} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_35870/1154517329.py", line 29, in objective
    model = lgb.train(
            ^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/engine.py", line 322, in train
    booster.update(fobj=fobj)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/b

KeyboardInterrupt: 

In [5]:
study.best_params

{'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 412,
 'max_depth': 443,
 'min_gain_to_split': 0.03926513379419705,
 'min_data_in_leaf': 244,
 'lambda_l1': 4.27643390441266,
 'lambda_l2': 5.0198218368454235,
 'num_boost_round': 1595}

In [6]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [7]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("custom_indicators/models/model_meta.txt")